<h1>Models</h1>

<h2>Data</h2>
I added the variable initial_cap_dict which you can modify to change the initial capacities and their lifetime. Beware that the capacity you entered should not exceed the value in "lifetime"

In [874]:




# Problem Data
T = 6  # Total number of years --> 2025, 2030, 2035
T_years = 5 # number of years represented by the mmodel years
println("From year 2020 to year $(2020+T*T_years)")
c_hours = 8760

# Base annual demand
base_annual_demand = 2e6 # MWh

# Penalty cost for unmet demand (€/MWh)
c_penalty = 1.0e6

load = load_profile_normalized * base_annual_demand
maximum(load)

# Technological parameters
# Technology Names
technologies = [:CHP, :Boiler, :HeatPump]


# Lifetime in "years" (or pairs of stages)
# e.g., lifetime[:CHP] = 2 means "2 years" of operational usage
lifetime_in_year = Dict(
    :CHP => 20,
    :Boiler => 30,     # e.g. 3-year lifetime => no retirement in 3-year horizon
    :HeatPump => 20,   # only 1-year lifetime => retires quickly
)

lifetime = Dict(
    :CHP       => Int(lifetime_in_year[:CHP]/T_years),
    :Boiler    => Int(lifetime_in_year[:Boiler]/T_years),     # e.g. 3-year lifetime => no retirement in 3-year horizon
    :HeatPump  => Int(lifetime_in_year[:HeatPump]/T_years),   # only 1-year lifetime => retires quickly
)




# You can set Initial Capacities (MW)
function create_initial_cap_dict(technologies::Vector{Symbol}, max_lifetime::Int)
    initial_cap_dict = Dict{Tuple{Symbol, Int}, Int}()
    
    for tech in technologies
        for life in 0:max_lifetime
            initial_cap_dict[(tech, life)] = 0
        end
    end
    
    return initial_cap_dict
end

technologies = [:CHP, :Boiler, :HeatPump]
Max_lifetime = Int(round(lifetime[:Boiler]))+1

initial_cap_dict = create_initial_cap_dict(technologies, Max_lifetime)
initial_cap_dict[:CHP,Int(round(10/T_years))] = 400
initial_cap_dict[:Boiler,Int(round(10/T_years))] = 350
initial_cap_dict[:HeatPump,Int(round(20/T_years))] = 100

# Print dictionary
println(initial_cap_dict)



# Operational Costs (€/MWh)
c_operational_cost = Dict(
    :CHP => 5.0,
    :Boiler => 0.75,
    :HeatPump => 1.1
)

# Investment Costs (€/MW)
c_investment_cost = Dict(
    :CHP => 2048825.0,
    :Boiler => 44528.0,
    :HeatPump => 591052.0
)

# Fixed O&M Costs (€/MW per year)
c_fixed_cost = Dict(
    :CHP => 43297.0,
    :Boiler => 1193.0,
    :HeatPump => 3316.0
)

# Maximum Additional Capacity (MW)
c_max_additional_capacity = Dict(
    :CHP => 500.0,
    :Boiler => 500.0,
    :HeatPump => 500.0
)

c_energy_carrier = Dict(
    :CHP => :nat_gas,
    :Boiler => :nat_gas,
    :HeatPump => :elec
)

# Efficiency (as a fraction)
c_efficiency_th = Dict(
    :CHP => 0.4,
    :Boiler => 0.9,
    :HeatPump => 3.0  # Coefficient of Performance (COP)
)

# Efficiency (as a fraction)
c_efficiency_el = Dict(
    :CHP => 0.5,
    :Boiler => 0.0,
    :HeatPump => 0.0  # Coefficient of Performance (COP)
)

# Emission factors (tCO2/MWh_th)
c_emission_fac = Dict(
    :nat_gas => 0.2,
    :elec => 0,
)


# Operational Costs (€/MWh)
c_opex_var = Dict(
    :CHP => 5.0,
    :Boiler => 0.75,
    :HeatPump => 1.1
)

# Carbon Price (€/t) for years
c_carbon_price = Dict(
    1 => 100,
    2 => 200,
    3 => 300,
    4 => 400,
    5 => 400,
    6 => 400,
    7 => 400,
    8 => 400,
)

# Function to get discount factor for stage t.
#   Each stage is 5 years. If t=1 => 2025, t=2 => 2030, etc.
#   We'll treat "year index" = t (1..T). 
discount_rate = 0.05
function discount_factor(t::Int, T_years::Int)
    # Each stage = 5 years
    exponent = T_years * (ceil(t / 2) - 1)
    return 1.0 / (1.0 + discount_rate)^exponent
end

salvage_fraction = 1

# Define the stochastic demand multipliers: +10%, no change, -10%
demand_multipliers = [1.1, 1.0, 0.9]

# Create a 3x3 identity matrix
I = Diagonal(ones(3))


# Define the matrices that will be repeated
mat1 = [0.3 0.5 0.2; 0.3 0.5 0.2; 0.3 0.5 0.2]  # The repeating matrix
I = Diagonal(ones(3))

# Initialize an empty array to store the matrices
transition_matrices = Array{Float64,2}[]

# Add the first two matrices manually
push!(transition_matrices, [1.0]')
push!(transition_matrices, [1.0]')


push!(transition_matrices, [0.3 0.5 0.2])  # Add the matrix [0.3 0.5 0.2; ...]
push!(transition_matrices, I)     # Add the identity matrix
# Dynamically add the alternating matrices based on T
for i in 1:T-2
    push!(transition_matrices, mat1)  # Add the matrix [0.3 0.5 0.2; ...]
    push!(transition_matrices, I)     # Add the identity matrix
end

println(transition_matrices)

the_iteration_limit = 100


# Log Normal distribution for the energy price
mean_price = 37.0  # Average energy price in currency units per unit of energy
price_volatility = 10.0  # Standard deviation
μ_normal = log(mean_price^2 / sqrt(price_volatility^2 + mean_price^2))
σ_normal = sqrt(log(1 + (price_volatility / mean_price)^2))
price_distribution = LogNormal(μ_normal, σ_normal)
num_price_scenarios = 10
price_quantiles = range(0.05, 0.95; length=num_price_scenarios)
price_values = quantile.(price_distribution, price_quantiles)
price_probabilities = pdf(price_distribution, price_values)
price_probabilities_normalized = price_probabilities / sum(price_probabilities)


From year 2020 to year 2050
Dict((:HeatPump, 3) => 0, (:CHP, 2) => 400, (:CHP, 7) => 0, (:Boiler, 1) => 0, (:HeatPump, 0) => 0, (:HeatPump, 7) => 0, (:HeatPump, 1) => 0, (:Boiler, 5) => 0, (:CHP, 5) => 0, (:Boiler, 4) => 0, (:CHP, 0) => 0, (:CHP, 3) => 0, (:Boiler, 0) => 0, (:HeatPump, 2) => 0, (:Boiler, 2) => 350, (:CHP, 4) => 0, (:Boiler, 6) => 0, (:CHP, 6) => 0, (:Boiler, 7) => 0, (:CHP, 1) => 0, (:HeatPump, 6) => 0, (:HeatPump, 4) => 100, (:Boiler, 3) => 0, (:HeatPump, 5) => 0)
[[1.0;;], [1.0;;], [0.3 0.5 0.2], [1.0 0.0 0.0; 0.0 1.0 0.0; 0.0 0.0 1.0], [0.3 0.5 0.2; 0.3 0.5 0.2; 0.3 0.5 0.2], [1.0 0.0 0.0; 0.0 1.0 0.0; 0.0 0.0 1.0], [0.3 0.5 0.2; 0.3 0.5 0.2; 0.3 0.5 0.2], [1.0 0.0 0.0; 0.0 1.0 0.0; 0.0 0.0 1.0], [0.3 0.5 0.2; 0.3 0.5 0.2; 0.3 0.5 0.2], [1.0 0.0 0.0; 0.0 1.0 0.0; 0.0 0.0 1.0], [0.3 0.5 0.2; 0.3 0.5 0.2; 0.3 0.5 0.2], [1.0 0.0 0.0; 0.0 1.0 0.0; 0.0 0.0 1.0]]


10-element Vector{Float64}:
 0.05517703795550454
 0.10613089994058338
 0.1313937458464863
 0.14183299861367246
 0.1414640921137261
 0.1323320758879828
 0.11558763902083811
 0.09183709042163361
 0.0612083595814727
 0.023036060618100176

<h1>Lifetime model</h1>
I added the salvage cost in the operational objective stage. It is equal to 0, except when you reach last stage (2*T)
Additonnally, I added +1 in lifetime to asset, and compute their power by summing from 1 to lifetime: in other words, the assets with lifetime+1 are not considered but will be considered at the next investment stage.

In [876]:

##############################################################################
# Create SDDP model
##############################################################################
# Create the Markovian policy graph
model = SDDP.MarkovianPolicyGraph(
    transition_matrices=transition_matrices,
    sense=:Min,
    lower_bound=0.0,  # Set the lower bound
    optimizer=Gurobi.Optimizer
) do sp, node
    t, demand_state = node

    # Variables
    @variables(sp, begin
        0 <= u_expansion_tech[tech in technologies] <= c_max_additional_capacity[tech]
        0 <= u_production_tech[tech in technologies, 1:n_typical_hours]
        0 <= u_unmet[1:n_typical_hours]
        0 <= x_annual_demand,                                                                       SDDP.State, (initial_value = base_annual_demand)
        0 <= cap[tech in technologies, lives in 0:Max_lifetime] <= c_max_additional_capacity[tech], SDDP.State, (initial_value = initial_cap_dict[tech,lives])
    end)

    ################### Investment stage (odd-numbered stages) ###################
    if t % 2 == 1
        @constraint(sp, x_annual_demand.out == x_annual_demand.in)

        for tech in technologies
            for lives in 0:Max_lifetime
                if lives == lifetime[tech] + 1
                    @constraint(sp, cap[tech,lives].out == cap[tech,lives].in + u_expansion_tech[tech]) #Possibility to expand with brand new
                else
                    @constraint(sp, cap[tech,lives].out == cap[tech,lives].in )
                end
                
            end
        end
        local df = discount_factor(t, T_years)
        investme_cost = @expression(sp, sum(c_investment_cost[tech] * u_expansion_tech[tech]     for tech in technologies))
        maintain_cost = @expression(sp, sum(cap[tech, lives].out * c_fixed_cost[tech]            for tech in technologies for lives in 1:lifetime[tech]))
        @stageobjective(sp, df * (investme_cost + T_years * maintain_cost))
                            

        ################### Operational stage (even-numbered stages) ###################
    else
        model_year = Int(ceil(t / 2))
        if t == 2
            d_annual = base_annual_demand
        else
            d_annual = demand_multipliers[demand_state] * x_annual_demand.in
        end

        @constraint(sp, [hour in 1:n_typical_hours], 
            sum(u_production_tech[tech, hour] for tech in technologies) + u_unmet[hour] == d_annual * typical_hours[hour][:value])

        @constraint(sp, [hour in 1:n_typical_hours, tech in technologies],
            u_production_tech[tech, hour] <= sum(cap[tech,lives].in for lives = 1:lifetime[tech])
        )

        # Aging capacity by removing one life.
        @constraint(sp, [tech in technologies, lives in 1:Max_lifetime],
            cap[tech,lives-1].out == cap[tech,lives].in
        )
        @constraint(sp, [tech in technologies],
            cap[tech,Max_lifetime].out == 0
        )
        
        
        # Update annual demand state
        @constraint(sp, x_annual_demand.out == d_annual)
        
        ### Stage objective (operational costs) ###
        SDDP.parameterize(sp, price_values, price_probabilities_normalized) do ω
            # discount factor for this stage
            local df = discount_factor(t, T_years)
            # Determine the relevant electricity purchase and sale price for the current model year
            local purch_elec_price = model_year <= 2 ? c_purch_elec_price_2030_typical : c_purch_elec_price_2050_typical
            local sale_elec_price = model_year <= 2 ? c_sale_elec_price_2030_typical : c_sale_elec_price_2050_typical

            
            # We treat (c_operational_cost[tech] + ω) as the total variable cost
            # operation_cost = sum((c_operational_cost[tech] + ω) * u_production_tech[tech, hour] 
            #     for tech in technologies  for hour in 1:n_typical_hours)
            # unmet_cost = sum(u_unmet[hour] * c_penalty * typical_hours[hour][:qty]  for hour in 1:n_typical_hours)   
            # sandwich_cost = 0
            # if t == 2*T
            #     sandwich_cost = sum(cap[tech,lives].out * c_investment_cost[tech] * lives/lifetime[tech] for tech in technologies for lives in 1:Max_lifetime)
            # end
            # @stageobjective(sp, operation_cost + unmet_cost - sandwich_cost)

            
            local expr_annual_cost = sum(
                (
                    sum(
                        c_opex_var[tech] * u_production_tech[tech, hour] #O&M
                        + (
                            (c_energy_carrier[tech] == :nat_gas ? ω : purch_elec_price[hour]) # random energy price | electricity price
                            + c_carbon_price[model_year] * c_emission_fac[c_energy_carrier[tech]] # carbon price
                            - c_efficiency_el[tech] * sale_elec_price[hour] # electricity revenue
                        )
                        * (u_production_tech[tech, hour] / c_efficiency_th[tech] # primary energy conversion
                        )
                        for tech in technologies
                    )
                    + u_unmet[hour] * c_penalty
                ) * typical_hours[hour][:qty]
                for hour in 1:n_typical_hours)
            local sandwich_cost = 0
            if t == 2*T
                sandwich_cost = sum(cap[tech,lives].out * c_investment_cost[tech] * lives/lifetime[tech] for tech in technologies for lives in 1:Max_lifetime)
            end
            # multiply by the 5-year block and discount factor
            @stageobjective(sp, df * (T_years * expr_annual_cost - sandwich_cost))
        end
    end
end

# Train the model
SDDP.train(model; iteration_limit=100)

# Retrieve results
println("Optimal Cost: ", SDDP.calculate_bound(model))



-------------------------------------------------------------------
         SDDP.jl (c) Oscar Dowson and contributors, 2017-25
-------------------------------------------------------------------
problem
  nodes           : 32
  state variables : 25
  scenarios       : 2.43000e+08
  existing cuts   : false
options
  solver          : serial mode
  risk measure    : SDDP.Expectation()
  sampling scheme : SDDP.InSampleMonteCarlo
subproblem structure
  VariableRef                             : [94, 94]
  AffExpr in MOI.EqualTo{Float64}         : [25, 35]
  AffExpr in MOI.LessThan{Float64}        : [30, 30]
  VariableRef in MOI.GreaterThan{Float64} : [69, 69]
  VariableRef in MOI.LessThan{Float64}    : [27, 28]
numerical stability report
  matrix range     [8e-06, 1e+00]
  objective range  [1e+00, 6e+09]
  bounds range     [5e+02, 5e+02]
  rhs range        [2e+01, 2e+06]
  - matrix range contains small coefficients
  - objective range contains large coefficients
Very large or small absolut

<h2>Simulations</h2>

In [884]:

# Simulation
simulations = SDDP.simulate(model, 1000, [:cap, :x_annual_demand, :u_production_tech, :u_expansion_tech, :u_unmet])

# Print simulation results
for t in 1:(T*2)
    sp = simulations[1][t]
    if t % 2 == 1  # Investment stages (odd stages)
        println("Year $(div(t + 1, 2)) - Investment Stage")
        for tech in technologies
            println("  Technology: $tech")
            println("    Expansion = ", value(sp[:u_expansion_tech][tech]))
            for lives in 0:Max_lifetime
                println("    Capacity $(String(tech)) $lives = ", value(sp[:cap][tech,lives].in))
            end
            
            println("    Expansion = ", value(sp[:u_expansion_tech][tech]))
        end
    else  # Operational stages
        println("Year $(div(t, 2)) - Operational Stage")
        println("   Demand_in = ", value(sp[:x_annual_demand].in))
        println("   Demand_out = ", value(sp[:x_annual_demand].out))
        
        total_production = sum(sum(value(sp[:u_production_tech][tech, hour] * typical_hours[hour][:qty]) for hour in 1:n_typical_hours) for tech in technologies)
        total_unmet = sum(value(sp[:u_unmet][hour] * typical_hours[hour][:qty]) for hour in 1:n_typical_hours)
        println("  Total Production = ", total_production)
        println("  Total Unmet Demand = ", total_unmet)
    end
end


plt = SDDP.SpaghettiPlot(simulations)

for tech in technologies
    SDDP.add_spaghetti(plt; title="Capacity_in_$tech") do data
        return sum(data[:cap][tech, lives].in for lives in 1:Max_lifetime)
    end
    SDDP.add_spaghetti(plt; title="Capacity_out_$tech") do data
        return sum(data[:cap][tech, lives].out for lives in 1:Max_lifetime)
    end
    SDDP.add_spaghetti(plt; title="Expansion_$tech") do data
        return data[:u_expansion_tech][tech]
    end
    SDDP.add_spaghetti(plt; title="Production_$tech") do data
        return sum(data[:u_production_tech][tech, :])
    end
end


SDDP.add_spaghetti(plt; title="Annual Demand_in") do data
    return data[:x_annual_demand].in
end
SDDP.add_spaghetti(plt; title="Annual Demand_out") do data
    return data[:x_annual_demand].out
end
SDDP.add_spaghetti(plt; title="Total Production") do data
    return sum(sum((data[:u_production_tech][tech, hour] * typical_hours[hour][:qty]) for hour in 1:n_typical_hours) for tech in technologies)
end
SDDP.add_spaghetti(plt; title="Unmet Demand") do data
    return sum(data[:u_unmet])
end

# SDDP.plot(plt, "spaghetti_plot.html")

Year 1 - Investment Stage
  Technology: CHP
    Expansion = 0.0
    Capacity CHP 0 = 0.0
    Capacity CHP 1 = 0.0
    Capacity CHP 2 = 400.0
    Capacity CHP 3 = 0.0
    Capacity CHP 4 = 0.0
    Capacity CHP 5 = 0.0
    Capacity CHP 6 = 0.0
    Capacity CHP 7 = 0.0
    Expansion = 0.0
  Technology: Boiler
    Expansion = 85.5047850886645
    Capacity Boiler 0 = 0.0
    Capacity Boiler 1 = 0.0
    Capacity Boiler 2 = 350.0
    Capacity Boiler 3 = 0.0
    Capacity Boiler 4 = 0.0
    Capacity Boiler 5 = 0.0
    Capacity Boiler 6 = 0.0
    Capacity Boiler 7 = 0.0
    Expansion = 85.5047850886645
  Technology: HeatPump
    Expansion = 369.8574979139811
    Capacity HeatPump 0 = 0.0
    Capacity HeatPump 1 = 0.0
    Capacity HeatPump 2 = 0.0
    Capacity HeatPump 3 = 0.0
    Capacity HeatPump 4 = 100.0
    Capacity HeatPump 5 = 0.0
    Capacity HeatPump 6 = 0.0
    Capacity HeatPump 7 = 0.0
    Expansion = 369.8574979139811
Year 1 - Operational Stage
   Demand_in = 2.0e6
   Demand_out = 2.0e